In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd

from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
import random
import datetime, os

In [ ]:
def data_generator(min_val = 1000, max_val = 2000, balance_data = True):
  xy = np.mgrid[min_val:max_val, min_val:max_val].reshape(2, -1)
  df = pd.DataFrame({'X' : xy[0], 'Y' : xy[1]})

  df['Z'] = (df['X'] != df['Y']).astype('float')
  df['X_minus_Y'] = df.X - df.Y

  if balance_data == True:
    df1 = df[df.X == df.Y]
    df2 = df[df.X != df.Y].sample(len(df1))
    df = pd.concat([df1, df2], ignore_index=True, sort=False).reset_index(drop = True)

  return df

def make_model():

  initial_learning_rate = 0.1
  lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True)
  opt = keras.optimizers.Adam(learning_rate=lr_schedule)
  #opt = tf.keras.optimizers.Adam(learning_rate)

  inputs = keras.Input(shape=(1,), name="xy")
  x = layers.Dense(128, activation=tf.keras.activations.relu, name="dense_1")(inputs)
  x = layers.Dense(64, activation=tf.keras.activations.relu, name="dense_2")(x)
  outputs = layers.Dense(1, name="predictions")(x)

  model = keras.Model(inputs=inputs, outputs=outputs)

  model.compile(
      optimizer=opt,  # Optimizer
      loss=keras.losses.MSE
  )

  return model

In [ ]:
TRAIN_BATCH = 32
TEST_BATCH = 4
#LR = 0.0001
EPOCHS = 200

random.seed(42)

In [ ]:
train = data_generator()
train_dataset = tf.data.Dataset.from_tensor_slices((train[['X_minus_Y']], train['Z']))
train_dataset = train_dataset.shuffle(buffer_size=2000, seed=42).batch(TRAIN_BATCH)

test = data_generator(min_val = 1, max_val = 10, balance_data=False)
test_dataset = tf.data.Dataset.from_tensor_slices((test[['X_minus_Y']], test['Z']))
test_dataset = test_dataset.batch(TEST_BATCH)

In [ ]:
model = make_model()

In [ ]:
print('Training model...')

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

model.fit(train_dataset, epochs=EPOCHS, shuffle=False, callbacks=[tensorboard_callback])

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
print("Evaluate")
model.evaluate(test_dataset)

In [ ]:
print('Make predictions')
predictions = model.predict(test[['X_minus_Y']])
test['predictions'] = predictions.reshape(-1)
test.sort_values('predictions').head(40)

In [ ]:
model.save('un_func_model')

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('un_func_model1') # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)